This is is basically your pom.xml or build.gradle.

In [1]:
%maven dev.langchain4j:langchain4j:0.27.1
%maven dev.langchain4j:langchain4j-core:0.27.1
%maven dev.langchain4j:langchain4j-open-ai:0.27.1

**Agent/Tool definition**
In this case we provide the main LLM tools, which are LLMs themselves.

For different agents we can even use different LLMs from different providers.

In [2]:
import dev.langchain4j.agent.tool.Tool;
import dev.langchain4j.service.UserMessage;
import dev.langchain4j.service.AiServices;
import dev.langchain4j.service.V;
import static java.time.Duration.ofSeconds;
import dev.langchain4j.model.chat.ChatLanguageModel;
import dev.langchain4j.model.openai.OpenAiChatModel;

static class BrainstormingAgent {

    interface Brainstorming {
        @UserMessage("ACT as a professional {{role}} and brainstorm on this problem statement: {{problemStatement}} " + 
                     "Give different possible solutions that make sense.")
        String brainstorm(@V("role") String role, @V("problemStatement") String problemStatement);
    }    

    static ChatLanguageModel brainstormingModel = OpenAiChatModel.builder()
                                                // Using LM Studio for local LLMs, so api key is not needed
                                                .apiKey("not-needed")
                                                // Using LM Studio for local LLMs (deploy a server with LM Studio)
                                                .baseUrl("http://localhost:1234/v1/")
                                                // Using LM Studio for local LLMs, so model name is not needed
                                                .modelName("not-needed")
                                                .temperature(0.3)
                                                .timeout(ofSeconds(60))
                                                .build();

    static Brainstorming brainstorming = AiServices.create(Brainstorming.class, brainstormingModel);

    @Tool("Brainstorming functon to brainstorm on a problem statement given a role")
    String brainstorm(String role, String problemStatement) {
        System.out.println("********************** brainstorming ***********************");
        return brainstorming.brainstorm(role, problemStatement);
    } 

    interface Evaluation {
        @UserMessage("ACT as a professional {{role}} and evaluate this problem Statement: {{problemStatement}} with these solutions: {{possibleSolutions}}. " +
                     "For each of the proposed solutions, evaluate their potential. Consider their pros and cons, " +
                     "initial effort needed, implementation difficulty, potential risks and how well they solve the problem. " +
                     "Give a score from 1-10 on how good the solution is to each option based on these factors.")
        String evaluate(@V("role") String role, @V("problemStatement") String problemStatement, @V("possibleSolutions") String possibleSolutions);
    }    

    static ChatLanguageModel evaluationModel = OpenAiChatModel.builder()
                                                // Using LM Studio for local LLMs, so api key is not needed
                                                .apiKey("not-needed")
                                                // Using LM Studio for local LLMs (deploy a server with LM Studio)
                                                .baseUrl("http://localhost:1234/v1/")
                                                // Using LM Studio for local LLMs, so model name is not needed
                                                .modelName("not-needed")
                                                .temperature(0.3)
                                                .timeout(ofSeconds(60))
                                                .build();

    static Evaluation evaluation = AiServices.create(Evaluation.class, evaluationModel);

    @Tool("Evaluation function to evaluate a problem statement with possible solutions given a role")
    String evaluatePossibleSolutions(String role, String problemStatement, String possibleSolutions) {
        System.out.println("********************** evaluation ***********************");
        return evaluation.evaluate(role, problemStatement, possibleSolutions);
    }
}



**Brain/prefrontal cortex definition**

This is the main LLM which will be used for the Tree-of-Thought coordination.

We can think of it as a prefrontal cortex of the assistant, which is responsible for thinking and planning.

Don't forget to replace "YOUR API KEY" with your actual OpenAI API key.

In [3]:
import static dev.langchain4j.model.openai.OpenAiChatModelName.GPT_3_5_TURBO;
import static dev.langchain4j.model.openai.OpenAiChatModelName.GPT_4_0125_PREVIEW;

ChatLanguageModel model = OpenAiChatModel.builder()
// your api key goes here
.apiKey("YOUR API KEY")
.modelName(GPT_4_0125_PREVIEW)
// GPT_3_5_TURBO is a less costly model but won't follow instructions as well
// .modelName(GPT_3_5_TURBO)
.temperature(0.3)
.timeout(ofSeconds(60))
.build();

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


The _@SystemMessage_ is the main entry point for the Tree of Thought approach.

If the main/"brain" AI model isn't performing according to your requirements, you try to adapt the _@SystemMessage_.

In [8]:

import dev.langchain4j.memory.chat.MessageWindowChatMemory;
import dev.langchain4j.service.SystemMessage;

interface TreeOfThoughtAssistant {

    @SystemMessage("Your task is to assist the user in their current task. You can ask questions to clarify the task, and then assist the user in completing the task. " +
                   "Your first question will ALWAYS be, which ROLE you as a helpful assistant should assume. " +
		           "If you have enough information, you ALWAYS HAVE TO start brainstorming using the ROLE as a context for your brainstorming. " + 
		           "For brainstorming, use 'brainstorming' tools which are available to you. " +
                   "After you have enough information, you ALWAYS HAVE TO start evaluating the possible solutions. It is extremely important and pivotal, that you use evalution functions available to you. " +
                   "For evaluation phase you can use 'evaluatePossibleSolutions' tools which are available to you. " +
                   "Communicate with the user so that the user does not know, that you as a assistant are performing other tasks behind the scenes, like brainstorming or evaluation. " +
                   "Based on the evaluations and scenarios, rank the solutions in the order from the best to the worst. Provide a justification for each ranking and offer any final thoughts " +
                   "or considerations for each solution.")
    String treeOfThoughtChat(String message);
}

TreeOfThoughtAssistant assistant = AiServices.builder(TreeOfThoughtAssistant.class)
                                .chatLanguageModel(model)
                                .tools(new BrainstormingAgent())
                                .chatMemory(MessageWindowChatMemory.withMaxMessages(10))
                                .build();

The main Tree of Thought LLM will reach out to its tools/agents to perform sub-task like brainstorming or evaluation.

Please keep in mind that this process may take a few minutes, so please be patient.

If you are using LM Studio, you can follow along with the logs in the LM Studio interface.

In [9]:
assistant.treeOfThoughtChat("I want you to assume a role of a philosopher. What is the meaning of life?")

********************** brainstorming ***********************
********************** evaluation ***********************


After evaluating the various philosophical perspectives on the meaning of life, here are the solutions ranked from best to worst, along with justifications for each ranking:

1. **Personal Growth and Transformation (9/10):** This perspective emphasizes self-awareness, self-reflection, and personal responsibility. It promotes positive change, self-actualization, and provides a framework for overcoming obstacles and achieving personal growth. Despite potential drawbacks like the possibility of narcissistic behavior, it offers a practical and fulfilling approach to finding life's meaning through personal development.

2. **Humanism's Perspective (8/10):** Humanism focuses on human values and experiences, offering an optimistic outlook that encourages empathy, compassion, and social engagement. It provides a practical framework for personal fulfillment, though it may overlook deeper existential questions.

3. **Social Connection and Contribution (8/10):** This solution highlights the impor